# NLP Fundamentals

* * * 

<div class="alert alert-success">  
    
### Learning Objectives 
    
* Learn common steps for preprocessing text data, as well as specific operations for preprocessing Reddit data.
* Know commonly used NLP packages and what they are capable of.
* Understand tokenizers, and how they have changed since the advent of Large Language Models.
* Learn how to convert text data into numbers through a Bag-of-Words approach.
* Understand the TF-IDF algorithm and how it complements the Bag-of-Words representation.
* Implement Bag-of-Words and TF-IDF using the `sklearn` package and understand its parameter settings.
* Use the numerical representations of text data to perform classification tasks.
</div>

### Icons Used in This Notebook
🔔 **Question**: A quick question to help you understand what's going on.<br>
💡 **Tip**: How to do something a bit more efficiently or effectively.<br>
⚠️ **Warning:** Heads-up about tricky stuff or common mistakes.<br>

### Sections
1. [Preprocessing](#section1)
2. [Tokenization](#section2)
3. [The Bag-of-Words Representation](#section3)
4. [Term Frequency-Inverse Document Frequency](#section4)
5. [Text Classification Using the TF-IDF Representation](#section5)

Today, we'll learn the building blocks for performing text analysis in Python using Reddit data from the popular subreddit r/AmItheAsshole (AITA). These techniques lie in the domain of Natural Language Processing (NLP). NLP is a field that deals with identifying and extracting patterns of language, primarily in written texts. Throughout the workshop, we'll interact with various packages for performing text analysis: starting from simple string methods to specific NLP packages, such as `nltk`, `spaCy`, and more recent ones on Large Language Models (`BERT`).

Now, let's have these packages properly installed before diving into the materials.

In [ ]:
# Uncomment the following lines to install packages/model
# %pip install NLTK
# %pip install transformers
# %pip install spaCy
# %pip install scikit-learn
!python -m spacy download en_core_web_sm

In [1]:
# Import necessary packages
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from string import punctuation
%matplotlib inline

<a id='section1'></a>

# Preprocessing

In the first part, we'll address the first step of text analysis. Our goal is to convert the raw, messy text data into a consistent format. This process is often called **preprocessing**, **text cleaning**, or **text normalization**.

You'll notice that at the end of preprocessing, our data is still in a format that we can read and understand. Later in this workshop, we will begin our foray into converting the text data into a numerical representation—a format that can be more readily handled by computers. 

🔔 **Question**: Let's pause for a minute to reflect on **your** previous experiences working on text data. 
- What is the format of the text data you have interacted with (plain text, CSV, or XML)?
- Where does it come from (structured corpus, scraped from the web, survey data)?
- Is it messy (i.e., is the data formatted consistently)?

## Common Processes

Preprocessing is not something we can accomplish with a single line of code. We often start by familiarizing ourselves with the data, and along the way, we gain a clearer understanding of the granularity of preprocessing we want to apply.

Typically, we begin by applying a set of commonly used processes to clean the data. These operations don't substantially alter the form or meaning of the data; they serve as a standardized procedure to reshape the data into a consistent format.

The following processes, for examples, are commonly applied to preprocess English texts of various genres. These operations can be done using built-in Python functions, such as `string` methods, and Regular Expressions. 
- Lowercase the text
- Remove punctuation marks
- Remove extra whitespace characters
- Remove stop words

After the initial processing, we may choose to perform task-specific processes, the specifics of which often depend on the downstream task we want to perform and the nature of the text data (i.e., its stylistic and linguistic features).  

Before we jump into these operations, let's take a look at our data!

### Import the Text Data

The text data we'll be working with is a CSV file containing posts from the Reddit subreddit r/AmItheAsshole (AITA), where people post about moral dilemmas and ask for judgment from the community. 

Let's read the file `aita_top_subs.csv` into dataframe with `pandas`.

In [2]:
# Import pandas
import pandas as pd

# Read the Reddit data
df = pd.read_csv('../../data/aita_top_subs.csv')

In [3]:
# Show the first five rows
df.head()

,idint,idstr,created,self,nsfw,author,title,url,selftext,score,...,num_comments,flair_text,flair_css_class,augmented_at,augmented_count,created_date,year,month,day_of_week,text_length
0,797709732,t3_d6xoro,1568998300,1.0,0.0,DarthCharizard,META: This sub is moving towards a value syste...,NaN,I’ve enjoyed reading and posting on this sub f...,80915.0,...,6215.0,META,NaN,NaN,NaN,2019-09-20 16:51:40,2019,9,Friday,3266.0
1,1472895100,t3_ocx94s,1625315782,1.0,0.0,OnlyInQuebec9,AITA for telling my wife the lock on my daught...,NaN,My brother in-law (Sammy) lost his home shortl...,80334.0,...,5318.0,Not the A-hole,not,NaN,NaN,2021-07-03 12:36:22,2021,7,Saturday,2664.0
2,664921441,t3_azvko1,1552322462,1.0,0.0,Renegadesrule33,"UPDATE, AITA for despising my mentally handica...",NaN,"I'm back like I said I would be,. My [original...",72776.0,...,1989.0,UPDATE,NaN,NaN,NaN,2019-03-11 16:41:02,2019,3,Monday,5437.0
3,855862814,t3_e5k3z2,1575392873,1.0,0.0,throwRA-fhfsveyary,AITA for pretending to get fired when customer...,NaN,I am a high schooler with a weekend job at a c...,63526.0,...,3645.0,Not the A-hole,not,NaN,NaN,2019-12-03 17:07:53,2019,12,Tuesday,2096.0
4,756636047,t3_cihc3z,1564233111,1.0,0.0,Thunderbear998,AITA for telling my extended family how many m...,NaN,We had a family dinner this evening. My family...,54132.0,...,5190.0,Everyone Sucks,ass,NaN,NaN,2019-07-27 13:11:51,2019,7,Saturday,1662.0


The dataframe has one row per Reddit post. Let's examine the columns and identify the main text column:

In [ ]:
# Show column names and basic info
# YOUR CODE HERE

Let's take a look at an example post. Pick the one at index 2.

In [ ]:
# Display first post
# YOUR CODE HERE

🔔 **Question**: What have you noticed about this formatting?

### Removing NaN

Remember how to grab all NaN (Not a Number) values in Pandas? Let's see if our dataset has any, and remove them if needed.

In [ ]:
# Get NaN values
# YOUR CODE HERE

### Removing Unuseful Rows

When you work with social media data, not every row will contain useful text. Sometimes you’ll find entries that look unusual — they aren’t empty, but they also aren’t really content you can analyze.

Take a close look at your dataset and see if you can find these cases. Once you’ve identified them, write code to remove those rows so that only meaningful text remains for analysis.

In [ ]:
# Find unuseful rows
# YOUR CODE HERE
# Remove rows where 'selftext' is '[removed]' or '[deleted]'


### Lowercasing

In classic NLP, we often don't work in contexts where we can properly utilize all linguistic information.

More often, the subsequent analysis we perform is **case-insensitive**. For instance, in frequency analysis, we want to account for various forms of the same word. Lowercasing the text data aids in this process and simplifies our analysis.

We can easily achieve lowercasing with the string method [`.lower()`](https://docs.python.org/3/library/stdtypes.html#str.lower); see [documentation](https://docs.python.org/3/library/stdtypes.html#string-methods) for more useful functions.

Let's apply it to an example:

In [ ]:
# Grab the previous example post's text for demonstration
# YOUR CODE HERE
text = df['selftext'].iloc[2]

# Convert it to lowercase
# YOUR CODE HERE
text = text.lower()

### Regex: Removing Whitespace

Sometimes we might come across texts with extraneous whitespace, such as spaces, tabs, and newline characters, which is particularly common when the text is scrapped from web pages. Before we dive into the details, let's briefly introduce Regular Expressions (regex) and the `re` package. 

Regular expressions are a powerful way of searching for specific string patterns in large corpora. They have an infamously steep learning curve, but they can be very efficient when we get a handle on them. Many NLP packages heavily rely on regex under the hood. Regex testers, such as [regex101](https://regex101.com), are useful tools in both understanding and creating regex expressions.

Our goal is not to provide a deep (or even shallow) dive into regex; instead, we want to expose you to them so that you are better prepared to do deep dives in the future!

Let's look at an example Reddit post with potential whitespace issues:

In [ ]:
# Create an example with extra whitespace (simulating messy Reddit data)
messy_reddit_post = """AITA for not going to my  sister's   wedding?

So here's the situation...    My sister is getting married and I was supposed to be in the wedding party.
But then she told me I couldn't bring my boyfriend    because "it's family only" which doesn't make sense."""

print("Original post:")
print(repr(messy_reddit_post))  # repr() shows the actual whitespace characters

First, look up what this piece of regex does: `r'\s+'`

In [ ]:
import re

# pattern in regex
blankspace_pattern = r'\s+'

# Write a replacement for the pattern identified
blankspace_repl = ' '

# Replace whitespace(s) with ' '
clean_text = re.sub(
    pattern=blankspace_pattern,
    repl=blankspace_repl,
    string=messy_reddit_post
)

print("Cleaned text:")
print(repr(clean_text))

### Removing Punctuation Marks

Sometimes we are only interested in analyzing **alphanumeric characters** (i.e., the letters and numbers), in which case we might want to remove punctuation marks. 

The `string` module contains a list of predefined punctuation marks. Let's print them out.

In [ ]:
# Load in a predefined list of punctuation marks
from string import punctuation
print(punctuation)

In [ ]:
def remove_punct(text):
    '''Remove punctuation marks in input text'''
    
    # Use string.punctuation to return text without punctuation
    text_no_punct = ''.join([char for char in text if char not in punctuation])

    return text_no_punct

In [ ]:
# Create an example Reddit post with punctuation
reddit_example = "AITA? My friend thinks I'm being 'dramatic' about this whole situation... What do you think?"

# Apply the function 
print(f"No punctuation: {remove_punct(reddit_example)}")

### Creating Placeholders
Older NLP models (bag-of-words, TF-IDF) treated every unique word as a feature. That meant URLs, usernames, and numbers exploded the vocabulary with noisy, one-off tokens. To keep models simple and useful, we often replaced these with placeholders like URL, DIGIT, or USER.

Write a function called `regex_remover` that that uses `re.sub` to:

* Replace the following patterns with placeholders:
    * URLs → ` URL `
    * Digits → ` DIGIT `
    * Subreddit mentions (r/subreddit) → ` SUBREDDIT `
    * User mentions (u/user or /u/user) → ` USER `

Here are some regex patterns to help you:
- URLs: `r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|www\.\S+'`
- Digits: `r'\d+'`
- Subreddits: `r'r/\w+'` 
- Users: `r'/u/\w+|u/\w+'`

In [ ]:
def regex_remover(text: str) -> str:
    """
    Replace URLs, digits, subreddits, and user mentions with placeholders.
    """
    
    # Step 1: replace URLs with ' URL '
    pattern = # URL regex pattern
    text = # re.sub(pattern, ' URL ', text

    # Step 2: replace digits with ' DIGIT '
    pattern = # Digit regex pattern
    text = # re.sub(pattern, ' DIGIT ', text)

    pattern = # Subreddit regex pattern
    text = # re.sub(pattern, ' SUBREDDIT ', text)

    # Step 4: replace user mentions with ' USER '
    pattern = # User mention regex pattern
    text = # re.sub(pattern, ' USER ', text)
    
    return text

# Example to test 
sample = "Check out http://reddit.com/r/AmItheAsshole 123 times! Thanks u/username"
print(regex_remover(sample))

As you can see, in classical NLP, preprocessing pipelines often normalize text aggressively (lowercasing, replacing URLs, digits, usernames). In LLM-based NLP, we usually leave text as-is.

<a id='section2'></a>

# Comparing Tokenizers

One of the most important steps in text analysis is tokenization. This is the process of breaking a long sequence of text into word tokens. With these tokens available, we are ready to perform word-level analysis. For instance, we can filter out tokens that don't contribute to the core meaning of the text.

In this section, we'll introduce how to perform tokenization using `nltk`, `spaCy`, and a Large Language Model (`bert`). The purpose is to expose you to different NLP packages, help you understand their functionalities, and demonstrate how to access key functions in each package.

### `nltk`

The first package we'll be using is called **Natural Language Toolkit**, or `nltk`.

In [ ]:
import nltk

In [ ]:
# Uncomment the following lines to install these modules
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('punkt')

NLTK is one of the earliest Python libraries for NLP. It provides simple, rule-based tools for tokenization, stemming, POS tagging, and more. Its tokenizers typically split text using whitespace, punctuation, or regex rules, which makes it easy to understand but sometimes brittle when handling messy, real-world text.

NLTK's tokenizer:
- Classic, rule/regex-based.
- Splits on whitespace/punctuation.

In [ ]:
# Load word_tokenize 
from nltk.tokenize import word_tokenize

# Print a Reddit example
reddit_text = "AITA for not wanting to go to my brother's wedding? UPDATE: I talked to my family."

# Apply the NLTK tokenizer
nltk_tokens = word_tokenize(reddit_text)
print(nltk_tokens)

In [ ]:
# Load predefined stop words from nltk
from nltk.corpus import stopwords

# Print the first 10 stopwords
stop = stopwords.words('english')
stop[:10]

### `spaCy`

spaCy is a more modern NLP library designed for speed and practical use. Its tokenizer combines rules with statistical models to handle edge cases more robustly. It also integrates linguistic features out of the box, like part-of-speech tags, lemmas, dependency parses, and named entities, making it a common choice for production-level NLP workflows.

First, look up how to get verbatim text of tokens using SpaCy. Look up documentation, ask an LLM, Google it -- whatever you want.

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

# Apply the pipeline to example Reddit text
doc = nlp(reddit_text)

# Get the verbatim texts of tokens
spacy_tokens = [token.text for token in doc]

# Compare with NLTK tokens
print(f"NLTK tokens: {nltk_tokens}")
print(f"spaCy tokens: {spacy_tokens}")

SpaCy adds linguistic knowledge (prefix/suffix rules, exceptions, POS tagging).

In [ ]:
from tabulate import tabulate

rows = []
for t in doc:
    rows.append([
        t.i,            # token index
        t.text,         # surface
        t.lemma_,       # lemma
        t.pos_,         # coarse POS
        t.tag_,         # fine POS
        t.dep_,         # dependency label
        t.head.text,    # syntactic head
        t.is_stop,      # stopword?
        t.shape_,       # orthographic shape
        (t.idx, t.idx + len(t.text))  # char span
    ])

print(tabulate(rows, headers=[word for word in reddit_text.split()]))

# sentence segmentation
print("\nSentences:")
for s in doc.sents:
    print("-", s.text)

## BERT Tokenizer

The BERT tokenizer is a subword tokenizer based on an algorithm called WordPiece. Instead of splitting text into whole words (like NLTK or spaCy), it breaks words into smaller units when the full word is not in its vocabulary. For example:
- spaCy: ["roommate"]
- BERT tokenizer: ["room", "##mate"]

This design solves two problems that word-level tokenizers have:
1. Out-of-vocabulary words: If a new word appears (“cryptomemes”), word-based tokenizers can’t handle it. Subword tokenization breaks it into familiar chunks (“crypto” + “##memes”).
2. Vocabulary size: Instead of keeping hundreds of thousands of word forms, models only need ~30,000 subword units. This makes training more efficient.

### bert-base-uncased tokenizer

In practice, we usually don’t train tokenizers from scratch. Instead, we use a pretrained tokenizer that was released together with a model. For example, calling `AutoTokenizer.from_pretrained("bert-base-uncased")` loads the WordPiece tokenizer associated with the BERT base model. 

This tokenizer has a fixed vocabulary of ~30k subword units that were learned during BERT’s pretraining. The important point is that the tokenizer and model are paired: the model only understands text that has been split into tokens exactly the way it saw during training.

In [ ]:
# WARNING: The following code cell may take a while to run because it downloads large models.

from transformers import AutoTokenizer, pipeline

# 1) Subword tokenization (BERT WordPiece)
tok = AutoTokenizer.from_pretrained("bert-base-uncased")

tokens = tok.tokenize(reddit_text)
print("Subword tokens:", tokens)

## Comparing Tokenization

Write **two** functions to tokenize and remove stop words from our full Reddit text data. 

1.	Using another tokenizer or preprocessing approach of their choice (e.g., spaCy, NLTK, regex, lowercasing, stopword removal, etc.)
2.	Using a modern LLM tokenizer (e.g., Hugging Face AutoTokenizer or OpenAI’s tiktoken)

In [ ]:
from nltk import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
stop = stopwords.words('english')

def og_preprocessor(raw_text):
    # text → tokenize → whitespace → URLS → stopwords 
    # YOUR CODE HERE

    # Lowercase
    # YOUR CODE HERE
    
    # Remove punctuation
    # YOUR CODE HERE
    
    # Tokenize
    # YOUR CODE HERE
    
    # Remove stopwords
    # YOUR CODE HERE
    return ' '.join(tokens)

In [ ]:
from transformers import AutoTokenizer, pipeline

def modern_preprocessor(raw_text):
    
    # text → subword tokenization
    # YOUR CODE HERE
    return ' '.join(tokens)

In [ ]:
# Process row index 2 from df.selftext using both preprocessors
text_example = df['selftext'].iloc[2]

print(og_preprocessor(text_example))
print(modern_preprocessor(text_example, stop))


🔔 **Question**: What assumptions about language do these tokenizers make? What gets lost? What gets preserved?

## Apply to data
FInally, let's apply our old-school preprocessor to our Reddit data -- we need the old approach for what we're about to do next.

In [ ]:
df['selftext_clean'] = df['selftext'].apply(og_preprocessor)

<a id='section3'></a>

# The Bag-of-Words Representation

Now we move beyond preprocessing to converting text into numerical representations. We'll explore one of the most straightforward ways to generate a numeric representation from text: the **bag-of-words** (BoW). 

At the heart of the bag-of-words approach lies the assumption that the frequency of specific tokens is informative about the semantics and sentiment underlying the text.

With a bag-of-words representation, we make heavy use of word frequency but not too much of word order.

## Document Term Matrix

Now let's implement the idea of bag-of-words using `CountVectorizer` from `sklearn`.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# A toy example containing four Reddit-like stories about different topics like weddings, work, and relationships
test_reddit = ['I am planning my wedding and need help with the guest list.',
               'My boss is being unreasonable about my work hours.',
               'I just got engaged and am so excited for the future!',
               'My partner and I are having communication issues.']

# Create a CountVectorizer object
vectorizer = CountVectorizer(stop_words='english')

# Fit and transform to create a DTM
test_count = vectorizer.fit_transform(test_reddit)

In [ ]:
# Create a DTM dataframe
test_dtm = pd.DataFrame(data=test_count.todense(),
                        columns=vectorizer.get_feature_names_out())
test_dtm

In [ ]:
# Getting the 15 most frequent words in the Reddit posts
top_words = test_dtm.sum().sort_values(ascending=False).head(15)
print("Top 15 most frequent words in Reddit posts:")
for word, freq in top_words.items():
    print(f"  {word}: {freq}")

### DTM for Reddit Posts

Let's create a Document Term Matrix for our reddit data, create a DTM dataframe, and grab most-frequent tokens.

In [ ]:
# Create a CountVectorizer object
# YOUR CODE HERE

In [ ]:
# Create a DTM dataframe
# YOUR CODE HERE

In [ ]:
# Most frequent tokens
# YOUR CODE HERE

<a id='section4'></a>

# Term Frequency-Inverse Document Frequency 

So far, we're relying on word frequency to give us information about a document. To remedy potential issues with very common words, we use a weighting scheme called **tf-idf (term frequency-inverse document frequency)**.

We can create a tf-idf DTM using `sklearn`'s `TfidfVectorizer`.

What happens is:
- First, we build a vocabulary of all the tokens that appear in our dataset.
- Each document (post) is turned into a row of numbers, one number per token in the vocabulary.
- These numbers are TF-IDF scores: they tell us which words are important within that post.
- Result = a big document–term matrix (DTM). This is the input to our model.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

test_reddit = ['I am planning my wedding and need help with the guest list.',
               'My boss is being unreasonable about my work hours.',
               'I just got engaged and am so excited for the future!',
               'My partner and I are having communication issues.']

# Create a tfidf vectorizer
vectorizer = TfidfVectorizer(lowercase=True,
                             stop_words='english',
                             max_features=None)

# Fit and transform 
tf_dtm = vectorizer.fit_transform(test_reddit)

# Create a tf-idf dataframe
tfidf = pd.DataFrame(tf_dtm.todense(),
                     columns=vectorizer.get_feature_names_out())
tfidf.head()

<a id='section5'></a>

## Text Classification Using the TF-IDF Representation

Now that we have a tf-idf representation of the text, we can do some simple classification!

In [ ]:
df.flair_text.unique()

In [ ]:
# 1) Keep only YTA / NTA and drop NaNs
mask = df["flair_text"].isin(["Asshole", "Not the A-hole"])
df_bin = df.loc[mask, ["selftext_clean", "flair_text"]].dropna()


In [ ]:
# Count the number of posts for each flair_text category
flair_counts = df['flair_text'].value_counts()
print(flair_counts[['Asshole', 'Not the A-hole']])

In [ ]:
# 2) Features/labels
X = df_bin["selftext_clean"].astype(str)
y = df_bin["flair_text"]

In [ ]:
from sklearn.model_selection import train_test_split

# 3) Train/test split (stratify keeps class balance)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

We can nowtrain a classifier using TF-IDF values, we do the following:
- We choose an algorithm (like Logistic Regression or Linear SVM).
- The classifier looks at the TF-IDF values across many posts classified as YTA or NTA.
- During training, it learns which words tend to signal which labels.
- Mathematically, this is stored as a set of coefficients: one number per vocabulary term, per class.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# 4) Pipeline: TF-IDF -> Logistic Regression
clf = Pipeline([
    ("tfidf", TfidfVectorizer(lowercase=True,
                              stop_words="english",
                              min_df=2,
                              max_df=0.95)),
    ("logreg", LogisticRegression(max_iter=1000))
])

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
# 5) Train and evaluate
clf.fit(X_train, y_train)
preds = clf.predict(X_test)

print("Accuracy:", round(accuracy_score(y_test, preds), 3))

How do coefficients work here? 
- We turn each post into a bag of TF-IDF features: one column per token, with a number that says how present and distinctive that token is in the post.
- Logistic Regression learns one weight per token. A positive weight pushes the prediction toward YTA (here classes[1]), a negative weight pushes toward NTA (here classes[0]).

For a post, the model computes a score:

$$\text{score} = b + \sum_{j} w_j \cdot \text{tfidf}_j$$

If the score is high (after passing through the sigmoid), it predicts YTA; if it’s low, NTA.

The bar charts show the tokens with the biggest weights in each direction. They’re not TF-IDF values; they’re model weights learned from the training data that indicate which tokens are most predictive for each class.

In [ ]:
# 6a) Visualize top tokens for YTA (largest positive weights)
import numpy as np
import matplotlib.pyplot as plt

vec = clf.named_steps["tfidf"]
logreg = clf.named_steps["logreg"]

feature_names = np.array(vec.get_feature_names_out())
weights = logreg.coef_[0]          # binary: positive -> classes_[1], negative -> classes_[0]
classes = logreg.classes_          # e.g., ['NTA', 'YTA']
top_k = 15

# Top toward YTA (largest positive weights)
yta_idx = np.argsort(weights)[-top_k:]
yta_tokens = feature_names[yta_idx]
yta_weights = weights[yta_idx]

# Sort for a clean horizontal chart
order = np.argsort(yta_weights)
yta_tokens = yta_tokens[order]
yta_weights = yta_weights[order]

plt.figure(figsize=(6, 5))
plt.barh(yta_tokens, yta_weights)
plt.title(f"Top tokens → {classes[1]} (higher = more YTA-like)")
plt.xlabel("Logistic regression weight")
plt.tight_layout()
plt.show()

In [ ]:
# 6b) Visualize top tokens for NTA (most negative weights)
nta_idx = np.argsort(weights)[:top_k]
nta_tokens = feature_names[nta_idx]
nta_weights = weights[nta_idx]   # these are negative numbers

# Sort so the strongest (most negative) appears at the bottom/top nicely
order = np.argsort(nta_weights)
nta_tokens = nta_tokens[order]
nta_weights = nta_weights[order]

plt.figure(figsize=(6, 5))
plt.barh(nta_tokens, nta_weights)
plt.title(f"Top tokens → {classes[0]} (more negative = more {classes[0]}-like)")
plt.xlabel("Logistic regression weight")
plt.tight_layout()
plt.show()

Make predictions
- For a new post, we compute TF-IDF values for its words.
- The classifier multiplies those numbers by its learned coefficients.
- The result is a score for each class, and the highest score is the predicted label.


In [ ]:
# Example new reddit post
new_post = ["I told my roommate she is kind of annoying, and now she's kind of upset at me."]

# Predict class
pred = clf.predict(new_post)[0]
proba = clf.predict_proba(new_post)[0]

print("Prediction:", pred)
print("Probabilities:", dict(zip(clf.classes_, np.round(proba, 3))))

<a id='section6'></a>

# Simple Sentiment Analysis with VADER

Now that we've explored classification, let's try a simpler approach to understanding the emotional tone of our Reddit posts using **VADER** (Valence Aware Dictionary and sEntiment Reasoner).

VADER is particularly well-suited for social media text because:
- It handles emojis, slang, and abbreviations
- It's very fast (no model training required)
- It's rule-based and interpretable
- It works well on short texts like social media posts

VADER gives us four scores for each text:
- **positive**: proportion of positive sentiment (0-1)
- **negative**: proportion of negative sentiment (0-1)
- **neutral**: proportion of neutral sentiment (0-1)
- **compound**: overall sentiment score (-1 to +1) ← **This is the one we'll use most**

In [ ]:
# Install VADER if needed
# !pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize VADER
analyzer = SentimentIntensityAnalyzer()

### Test VADER on Example Posts

Let's see how VADER scores different types of AITA posts:

In [ ]:
# Test on a few example posts with different emotional tones
examples = [
    "I love my family so much! They're amazing and supportive. 😊",
    "My roommate is the worst. He never cleans and eats my food. I hate this.",
    "I went to the store and bought groceries."
]

print("Example Sentiment Scores:\n" + "="*60)
for text in examples:
    scores = analyzer.polarity_scores(text)
    print(f"\nText: {text}")
    print(f"Scores: {scores}")
    print(f"Overall (compound): {scores['compound']:.3f}")

### Apply VADER to Our Reddit Data

Now let's analyze the sentiment of all our AITA posts:

In [ ]:
# Apply VADER to all posts
df['sentiment_scores'] = df['selftext'].apply(lambda x: analyzer.polarity_scores(str(x)))

# Extract the compound score (overall sentiment)
df['sentiment_compound'] = df['sentiment_scores'].apply(lambda x: x['compound'])

# Also extract individual components
df['sentiment_pos'] = df['sentiment_scores'].apply(lambda x: x['pos'])
df['sentiment_neg'] = df['sentiment_scores'].apply(lambda x: x['neg'])
df['sentiment_neu'] = df['sentiment_scores'].apply(lambda x: x['neu'])

# Show examples
print("Sample sentiment scores:")
df[['selftext', 'sentiment_compound', 'sentiment_pos', 'sentiment_neg']].head()

### Categorize Sentiment

We can convert the compound scores into simple categories:

In [ ]:
def categorize_sentiment(compound_score):
    """Categorize sentiment based on compound score"""
    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

df['sentiment_category'] = df['sentiment_compound'].apply(categorize_sentiment)

# Show distribution
print("Sentiment distribution across AITA posts:")
print(df['sentiment_category'].value_counts())
print(f"\nPercentages:")
print(df['sentiment_category'].value_counts(normalize=True).round(3) * 100)

### Visualize Sentiment Distribution

In [ ]:
# Create visualizations
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of compound scores
axes[0].hist(df['sentiment_compound'], bins=30, edgecolor='black', alpha=0.7)
axes[0].axvline(x=0, color='red', linestyle='--', alpha=0.5, label='Neutral')
axes[0].axvline(x=0.05, color='green', linestyle='--', alpha=0.5, label='Positive threshold')
axes[0].axvline(x=-0.05, color='orange', linestyle='--', alpha=0.5, label='Negative threshold')
axes[0].set_xlabel('Sentiment Compound Score')
axes[0].set_ylabel('Number of Posts')
axes[0].set_title('Distribution of Sentiment Scores')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Bar chart of categories
sentiment_counts = df['sentiment_category'].value_counts()
colors = {'positive': 'green', 'neutral': 'gray', 'negative': 'red'}
bar_colors = [colors[cat] for cat in sentiment_counts.index]

axes[1].bar(sentiment_counts.index, sentiment_counts.values, color=bar_colors, alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Sentiment Category')
axes[1].set_ylabel('Number of Posts')
axes[1].set_title('Sentiment Categories')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### Compare Sentiment with AITA Judgments

Let's see if sentiment correlates with whether someone was judged as the asshole:

In [ ]:
# Compare sentiment by flair (judgment)
# Filter to posts with YTA or NTA judgments
df_judged = df[df['flair_text'].isin(['Asshole', 'Not the A-hole'])].copy()

# Calculate average sentiment by judgment
sentiment_by_judgment = df_judged.groupby('flair_text')['sentiment_compound'].agg(['mean', 'std', 'count'])
print("Average sentiment by AITA judgment:")
print(sentiment_by_judgment)

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))

df_judged.boxplot(column='sentiment_compound', by='flair_text', ax=ax)
ax.set_xlabel('AITA Judgment')
ax.set_ylabel('Sentiment Compound Score')
ax.set_title('Sentiment Distribution by Judgment')
ax.axhline(y=0, color='red', linestyle='--', alpha=0.5, label='Neutral')
plt.suptitle('')  # Remove default title
ax.legend()
plt.tight_layout()
plt.show()

### Examine Extreme Examples

Let's look at the most positive and most negative posts:

In [ ]:
# Most positive post
most_positive_idx = df['sentiment_compound'].idxmax()
print("Most POSITIVE post:")
print(f"Score: {df.loc[most_positive_idx, 'sentiment_compound']:.3f}")
print(f"Text preview: {df.loc[most_positive_idx, 'selftext'][:300]}...")
print("\n" + "="*80 + "\n")

# Most negative post
most_negative_idx = df['sentiment_compound'].idxmin()
print("Most NEGATIVE post:")
print(f"Score: {df.loc[most_negative_idx, 'sentiment_compound']:.3f}")
print(f"Text preview: {df.loc[most_negative_idx, 'selftext'][:300]}...")

🔔 **Question**: What patterns do you notice in the sentiment distribution? Does sentiment align with AITA judgments? Why or why not?

💡 **Tip**: Remember that VADER measures the emotional tone of the *writing*, not whether the person was right or wrong. Someone can describe a situation calmly (neutral sentiment) even if they were the asshole, or describe it very negatively even if they weren't!

<div class="alert alert-success">

## ❗ Key Points

* Preprocessing includes multiple steps, some of them are more common to text data regardlessly, and some are task-specific. Reddit data requires specific preprocessing for URLs, user mentions, and subreddit references.
* Both `nltk` and `spaCy` could be used for tokenization and stop word removal. The latter is more powerful in providing various linguistic annotations. 
* A Bag-of-Words representation is a simple method to transform our text data to numbers. It focuses on word frequency but not word order. 
* A TF-IDF representation is a step further; it also considers if a certain word distinctively appears in one document or occurs uniformally across all documents. 
* With a numerical representation, we can perform a range of text classification task, such as post length categorization or potentially sentiment analysis with labeled data.
* Reddit data provides rich opportunities for text analysis due to its conversational nature and community-specific language patterns.

</div>